In [1]:
import warnings

warnings.filterwarnings("ignore")

In [2]:
from langchain import hub

prompt_agent = hub.pull('hwchase17/openai-tools-agent')

prompt_agent

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(in

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

llm.invoke("HI")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-b6a6615c-2f91-4cb4-a5e2-e827305df2f7-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17})

Agent = LLM + Tools

In [4]:
import subprocess
from langchain_core.tools import tool


@tool
def list_files():
    """List files in current directory"""
    return subprocess.run(["ls"], stdout=subprocess.PIPE).stdout.decode("utf-8")


list_files.invoke({"input": ""})

'1.0-langchain-basics.ipynb\n1.1-langchain-simple-chains.ipynb\n1.2-langchain-runnables-and-lcel.ipynb\n1.3-langchain-simple-agent.ipynb\n2.0-autogen-building-agents.ipynb\n2.1-autogen-building-research-assistant.ipynb\n3.0-langgraph-building-stateful-applications.ipynb\n3.1-langgraph-buid-research-assistant.ipynb\n4.0-rag-agent-langgraph.ipynb\n4.1-autogen-multi-agent-research-workflows.ipynb\nAutogen Demo Notebook.ipynb\nOAI_CONFIG_LIST\nTool Use.ipynb\nagentic-rag-langgraph.ipynb\n\x1b\x1bassets-resources\x1b[m\x1b[m\n\x1b\x1bdemo-notebooks\x1b[m\x1b[m\nreport-olympics-2024.md\n'

In [5]:
llm_with_tool = llm.bind_tools([list_files])

llm_with_tool

RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x11605cfd0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x116afb550>, root_client=<openai.OpenAI object at 0x11649f010>, root_async_client=<openai.AsyncOpenAI object at 0x116d1bd50>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_api_base='https://api.openai.com/v1', openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'list_files', 'description': 'List files in current directory', 'parameters': {'type': 'object', 'properties': {}}}}]})

In [6]:
llm_with_tool.invoke("List files in current directory")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_xdTc3oLiYznZGrDXjuIoIv1d', 'function': {'arguments': '{}', 'name': 'list_files'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 42, 'total_tokens': 52}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b573335b-3211-4eb7-ab8c-4ca21efc81db-0', tool_calls=[{'name': 'list_files', 'args': {}, 'id': 'call_xdTc3oLiYznZGrDXjuIoIv1d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 10, 'total_tokens': 52})

In [7]:
dictionary_with_tool_names = {
    "list_files": list_files,
}

output = llm_with_tool.invoke("List files in current directory")

if output.tool_calls:
    for tool_call in output.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call['args']
        tool_func = dictionary_with_tool_names[tool_name]
        print(tool_func.invoke(tool_args))

1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-research-workflows.ipynb
Autogen Demo Notebook.ipynb
OAI_CONFIG_LIST
Tool Use.ipynb
agentic-rag-langgraph.ipynb
assets-resources
demo-notebooks
report-olympics-2024.md



Agent = LLM + tools

In [8]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [10]:
tools = [list_files]

agent = create_tool_calling_agent(llm, tools, prompt_agent)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
agent_executor.invoke({'input': "List files in current directory"})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-research-workflows.ipynb
Autogen Demo Notebook.ipynb
OAI_CONFIG_LIST
Tool Use.ipynb
agentic-rag-langgraph.ipynb
assets-resources
demo-notebooks
report-olympics-2024.md
Here are the files in the current directory:

1. 1.0-langchain-basics.ipynb
2. 1.1-langchain-simple-chains.ipynb
3. 1.2-langchain-runnables-and-lcel.ipynb
4. 1.3-langchain-simple-agent.ipynb
5. 2.0-autogen-building-agents.ipynb
6. 2.1-autogen-building-research-assistant.ipynb
7. 3.0-langgraph-building-stateful-applications.ipynb
8. 3.1-langgraph-buid-research-assistant.ipynb
9. 4.0-rag-agent

{'input': 'List files in current directory',
 'output': 'Here are the files in the current directory:\n\n1. 1.0-langchain-basics.ipynb\n2. 1.1-langchain-simple-chains.ipynb\n3. 1.2-langchain-runnables-and-lcel.ipynb\n4. 1.3-langchain-simple-agent.ipynb\n5. 2.0-autogen-building-agents.ipynb\n6. 2.1-autogen-building-research-assistant.ipynb\n7. 3.0-langgraph-building-stateful-applications.ipynb\n8. 3.1-langgraph-buid-research-assistant.ipynb\n9. 4.0-rag-agent-langgraph.ipynb\n10. 4.1-autogen-multi-agent-research-workflows.ipynb\n11. Autogen Demo Notebook.ipynb\n12. OAI_CONFIG_LIST\n13. Tool Use.ipynb\n14. agentic-rag-langgraph.ipynb\n15. assets-resources (directory)\n16. demo-notebooks (directory)\n17. report-olympics-2024.md'}

In [12]:
agent_executor.invoke({'input': "What kind of files are in the current directory?"})



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`


1.0-langchain-basics.ipynb
1.1-langchain-simple-chains.ipynb
1.2-langchain-runnables-and-lcel.ipynb
1.3-langchain-simple-agent.ipynb
2.0-autogen-building-agents.ipynb
2.1-autogen-building-research-assistant.ipynb
3.0-langgraph-building-stateful-applications.ipynb
3.1-langgraph-buid-research-assistant.ipynb
4.0-rag-agent-langgraph.ipynb
4.1-autogen-multi-agent-research-workflows.ipynb
Autogen Demo Notebook.ipynb
OAI_CONFIG_LIST
Tool Use.ipynb
agentic-rag-langgraph.ipynb
assets-resources
demo-notebooks
report-olympics-2024.md
The current directory contains the following files:

1. Jupyter Notebooks (.ipynb):
   - 1.0-langchain-basics.ipynb
   - 1.1-langchain-simple-chains.ipynb
   - 1.2-langchain-runnables-and-lcel.ipynb
   - 1.3-langchain-simple-agent.ipynb
   - 2.0-autogen-building-agents.ipynb
   - 2.1-autogen-building-research-assistant.ipynb
   - 3.0-langgraph-building-stateful-applications.ipynb
   - 3.1-la

{'input': 'What kind of files are in the current directory?',
 'output': 'The current directory contains the following files:\n\n1. Jupyter Notebooks (.ipynb):\n   - 1.0-langchain-basics.ipynb\n   - 1.1-langchain-simple-chains.ipynb\n   - 1.2-langchain-runnables-and-lcel.ipynb\n   - 1.3-langchain-simple-agent.ipynb\n   - 2.0-autogen-building-agents.ipynb\n   - 2.1-autogen-building-research-assistant.ipynb\n   - 3.0-langgraph-building-stateful-applications.ipynb\n   - 3.1-langgraph-buid-research-assistant.ipynb\n   - 4.0-rag-agent-langgraph.ipynb\n   - 4.1-autogen-multi-agent-research-workflows.ipynb\n   - Autogen Demo Notebook.ipynb\n   - Tool Use.ipynb\n   - agentic-rag-langgraph.ipynb\n\n2. Other files:\n   - OAI_CONFIG_LIST\n   - report-olympics-2024.md\n\n3. Directories:\n   - assets-resources\n   - demo-notebooks'}

In [ ]:
import os

os.environ["TAVILY_API_KEY"] = 'YOUR TAVILY API KEY'

In [14]:
from langchain_community.tools import TavilySearchResults

In [15]:
search_tool = TavilySearchResults(
    max_results = 5,
    search_depth="advanced",
    include_answer=True,
)

search_tool.invoke('What is the capital of France?')

[{'url': 'https://www.britannica.com/facts/Paris',
  'content': 'Paris is the capital of France, located in the north-central part of the country. It is a major centre of commerce, culture, and tourism, with famous landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame de Paris.'},
 {'url': 'https://simple.wikipedia.org/wiki/Capital_of_France',
  'content': 'Learn about the history and current status of the capital of France, which is Paris. Find out which other cities have been the national capital in different periods of time.'},
 {'url': 'https://simple.wikipedia.org/wiki/Paris',
  'content': 'Events[change | change source]\nRelated pages[change | change source]\nReferences[change | change source]\nOther websites[change | change source]\nAmsterdam, Netherlands\xa0·\nAthens, Greece\xa0·\nBerlin, Germany\xa0·\nBratislava, Slovakia\xa0·\nBrussels, Belgium\xa0·\nBucharest, Romania\xa0·\nBudapest, Hungary\xa0·\nCopenhagen, Denmark\xa0·\nDublin, Republic of Ireland\xa0·\nHe

In [16]:
@tool
def create_file(file_path: str, content: str):
    """Create a file with the given content"""
    with open(file_path, 'w') as f:
        f.write(content)
    return f"File created at {file_path} with content: {content}"

tools = [create_file, search_tool]

In [17]:
simple_research_agent = create_tool_calling_agent(llm, tools, prompt_agent)

simple_research_agent_executor = AgentExecutor(agent=simple_research_agent, tools=tools, verbose=True)

simple_research_agent_executor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_

In [18]:
from IPython.display import Markdown
output_report = simple_research_agent_executor.invoke({'input': 'Write a summarized report about the main highlights of the Paris Olympics 2024 and save it locally as report-olympics-2024.md'})

Markdown(output_report['output'])



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'Paris Olympics 2024 highlights'}`


[{'url': 'https://olympics.com/en/paris-2024/videos/list', 'content': 'Watch the latest highlights from the Paris 2024 Olympic Games, featuring all sports, ceremonies and stars. See the best moments of rugby, judo, boxing, basketball, cycling, wrestling, water polo and more.'}, {'url': 'https://olympics.com/en/paris-2024/videos/list/highlights?team=us', 'content': "Watch the latest highlights from Paris 2024 and find out who is making headlines at the Summer Olympics (Jul 26-Aug 11, 2024). ... Olympic Games Paris 2024. The Men's Water polo Bronze Medal Match was held on 11/08/2024 at La Defense Arena, Paris. The United States of America claimed the Bronze with an 11:8 victory over Hungary. Load More Next ..."}, {'url': 'https://www.nbcolympics.com/videos', 'content': 'Watch highlights, interviews, documentaries and more from the 2024 Paris Olympic Games.'

The report on the highlights of the Paris Olympics 2024 has been successfully created and saved as `report-olympics-2024.md`. If you need any further assistance or modifications, feel free to ask!

In [19]:
with open('report-olympics-2024.md', 'r') as f:
    content_report = f.read()
    
Markdown(content_report)

# Highlights of the Paris Olympics 2024

The Paris Olympics 2024, scheduled from July 26 to August 11, 2024, promises to be a spectacular event showcasing athletic excellence and cultural celebration. Here are the main highlights:

## Key Events and Sports
- **Diverse Sports**: The Olympics will feature a wide range of sports including rugby, judo, boxing, basketball, cycling, wrestling, and water polo.
- **Men's Water Polo**: Notably, the Men's Water Polo Bronze Medal Match will take place at La Defense Arena, where the USA aims for a podium finish.

## Star Athletes
- **Steph Curry**: The NBA superstar is set to compete for his first Olympic gold medal, having made headlines with his impressive performance, including 22 three-pointers in six games.

## Viewing and Coverage
- **Live Events and Highlights**: Fans can catch live events, highlights, and interviews through various platforms, ensuring comprehensive coverage of the games.

## Cultural Significance
- The Paris Olympics will not only focus on sports but also celebrate the rich culture and history of France, making it a unique experience for athletes and spectators alike.

For more detailed highlights and updates, visit the official Olympic website and other sports news platforms.